In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using QPControl
using LCPSim
using MeshCat
using MeshCatMechanisms
using LearningMPC
using LearningMPC.Models
using RigidBodySim
using Gurobi
using SimpleQP

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/QPControl.ji for module QPControl.


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [233]:
vis = Visualizer()
open(vis)

Listening on 0.0.0.0:8704...


INFO: Serving MeshCat visualizer at http://127.0.0.1:8704
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [331]:
Δt = 0.002

0.002

In [338]:
control_model = BoxAtlas(add_contacts=false)
sim_model = BoxAtlas(add_contacts=true)
# position_bounds(findjoint(mechanism(control_model), "floating_base")) .= RigidBodyDynamics.Bounds(-Inf, Inf)
position_bounds(findjoint(mechanism(sim_model), "floating_base")) .= RigidBodyDynamics.Bounds(-Inf, Inf)

delete!(vis)
mvis = MechanismVisualizer(sim_model, vis)

lqrsol = let params = MPCParams(control_model)
    params.Δt = 0.05
    LQRSolution(control_model, params)
end

(::LQRSolution) (generic function with 2 methods)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [339]:
mvis2 = MechanismVisualizer(mechanism(sim_model), Skeleton(randomize_colors=true))
open(mvis2)
copy!(mvis2, nominal_state(sim_model))

Listening on 0.0.0.0:8714...


INFO: Serving MeshCat visualizer at http://127.0.0.1:8714
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [340]:
state = nominal_state(sim_model)
set_velocity!(state, 
    findjoint(mechanism(sim_model), "floating_base"), 
    [-0.4, 0, 0])
problem = LearningMPC.simulation_problem(state, lqrsol, Δt, 5.0)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

In [341]:
mpc = QPControl.MPCController{QPControl.ContactPoint{4}}(
    mechanism(sim_model),
    GurobiOptimizer(OutputFlag=0, TimeLimit=1)
)
stage = addstage!(mpc, Δt)
floor = HalfSpace3D(Point3D(root_frame(mechanism(sim_model)), 0., 0, 0),
    FreeVector3D(root_frame(mechanism(sim_model)), 0., 0, 1))

for bodyname in ["r_foot_sole", "l_foot_sole"]
    body = findbody(mechanism(sim_model), bodyname)
    position = Point3D(default_frame(body), 0., 0, 0)
    μ = 0.4
    contact = addcontact!(mpc, stage, position, floor, μ, 
        QPControl.LCPContact(separation_atol=1e-3),
#         QPControl.BooleanContact(separation_atol=1e-3)
    )
    contact.maxnormalforce[] = 1e6
end

objective = 0
for stage in mpc.stages
    x̄ = @expression vcat(stage.q, stage.v) - lqrsol.x0
    ū = @expression stage.u - lqrsol.u0
#     ū = stage.u
#     ū = stage.v̇
    objective = @expression objective + x̄' * lqrsol.Q * x̄ 
    objective = @expression objective + ū' * lqrsol.R * ū
    for contact in stage.contacts
        contact.weight[] = 1e-9
        objective = @expression objective + QPControl.objectiveterm(contact, mpc.qpmodel)
    end
end
x̄ = @expression vcat(last(mpc.stages).q, last(mpc.stages).v) - lqrsol.x0
objective = @expression objective + x̄' * lqrsol.S * x̄;
@objective mpc.qpmodel Minimize objective

Academic license - for non-commercial use only


In [342]:
state = nominal_state(sim_model)
set_velocity!(state, findjoint(mechanism(sim_model), "floating_base"), 
    [-0.4, 0, 0])
problem = LearningMPC.simulation_problem(state, mpc, Δt, 5.0; damping_kd=0.0)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")

In [330]:
setanimation!(mvis, sol)

true

In [200]:
function simulate_lcp!(state::MechanismState, mpc::QPControl.MPCController, t_final=1.0)
    # Simulate the system by repeatedly running the MPC controller
    # and then updating the state based on the expected state
    # at the end of the first MPC stage.
    ts = Float64[]
    qs = Vector{Float64}[]
    vs = Vector{Float64}[]
    τ = similar(velocity(state))
    for t in 0 : (mpc.stages[1].Δt) : t_final
        push!(ts, t)
        push!(qs, configuration(state))
        push!(vs, velocity(state))
        mpc(τ, t, state)
        if any(isnan, τ)
            break
        end
        # Assume the dynamics behave exactly as modeled in the optimization
        set_configuration!(state, SimpleQP.value.(mpc.qpmodel, mpc.stages[1].q))
        set_velocity!(state, SimpleQP.value.(mpc.qpmodel, mpc.stages[1].v))
        
#         set_configuration!(mpc.state, configuration(state))
#         set_velocity!(mpc.state, velocity(state))
#         for contact in mpc.stages[1].contacts
#             if string(contact.position.frame) == "after_pelvis_to_l_foot_sole_extension"
#                 point_world = transform(mpc.state, contact.position, root_frame(mpc.state.mechanism))
#                 Jp = point_jacobian(mpc.state, 
#                     path(mpc.state.mechanism, root_body(mpc.state.mechanism), 
#                         body_fixed_frame_to_body(mpc.state.mechanism, contact.position.frame)),
#                     point_world)
#                 v_world = point_velocity(Jp, velocity(mpc.state))
#                 @show point_world.v
#                 @show v_world.v
#                 @show SimpleQP.value.(mpc.qpmodel, contact.force_local.v)
#             end
#         end
        
    end
    ts, qs, vs
end

simulate_lcp! (generic function with 2 methods)

In [201]:
state = nominal_state(sim_model)
set_velocity!(state, 
    findjoint(mechanism(sim_model), "floating_base"), 
    [-0.4, 0, 0])
# set_configuration!(state, 
#     findjoint(mechanism(sim_model), "floating_base"), 
#     [0.5, 1.0, 0])
ts, qs, vs = simulate_lcp!(state, mpc, 1.0)
setanimation!(mvis, ts, qs, fps=round(Int, 1/Δt))

true

In [204]:
lcpsim_mpc = let params = MPCParams(control_model)
    params.horizon = 1
    params.Δt = Δt
    LearningMPC.MPCController(control_model, params, lqrsol, [lqrsol])
end

(::MPCController) (generic function with 2 methods)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [211]:
function simulate_lcpsim_mpc!(state::MechanismState, mpc::LearningMPC.MPCController, t_final=1.0)
    # Simulate the system by repeatedly running the MPC controller
    # and then updating the state based on the expected state
    # at the end of the first MPC stage.
    ts = Float64[]
    qs = Vector{Float64}[]
    vs = Vector{Float64}[]
    τ = similar(velocity(state))
    for t in 0 : (mpc.params.Δt) : t_final
        push!(ts, t)
        push!(qs, configuration(state))
        push!(vs, velocity(state))
        set_configuration!(mpc.scratch_state, configuration(state))
        set_velocity!(mpc.scratch_state, velocity(state))
        results = LearningMPC.run_mpc(mpc.scratch_state, mpc.env, mpc.params, mpc.cost, mpc.warmstart_controllers)
        if isnull(results.lcp_updates)
            break
        end
        updates = get(results.lcp_updates)
        set_configuration!(state, configuration(updates[1].state))
        set_velocity!(state, velocity(updates[1].state))
    end
    ts, qs, vs
end

simulate_lcpsim_mpc! (generic function with 2 methods)

In [209]:
state = nominal_state(sim_model)
set_velocity!(state, findjoint(mechanism(sim_model), "floating_base"), 
    [-0.4, 0, 0])
problem = LearningMPC.simulation_problem(state, lcpsim_mpc, Δt, 1.0; damping_kd=0.0)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

In [212]:
state = nominal_state(sim_model)
set_velocity!(state, 
    findjoint(mechanism(sim_model), "floating_base"), 
    [-0.4, 0, 0])
ts, qs, vs = simulate_lcpsim_mpc!(state, lcpsim_mpc, 1.0)
setanimation!(mvis, ts, qs, fps=round(Int, 1/Δt))

true

LoadError: [91mPrincipal inertias [0.014, 0.025, 0.039] do not satisfy the triangle inequalities, so the equivalent inertial ellipsoid is not well-defined.[39m